# Face Detector | Mask Detector with Cascade

In [ ]:
# Importing required libraries
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Define class for Mask and Face Detector
class MaskAndFaceDetector:
    # Initializer/Constructor for the class
    def __init__(self):
        # Load mask classifier model
        self.mask_classifier = self._load_trained_model('model2-013.model')
        # Load face classifier model
        self.face_classifier = self._load_trained_model('face_detection_model.h5')
        
        # Define labels dictionary
        self.label_dict = {0:'No Mask', 1:'Mask'}
        # Define colors dictionary to represent mask/no mask situation
        self.color_dict = {0:(0,0,255), 1:(0,255,0)}
        
        # Define scale for face detection
        self.scale = 4
        # Load pre-trained face detector model from OpenCV
        self.face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    # Method to load model from the path
    def _load_trained_model(self, model_path):
        return load_model(model_path)

    # Method to prepare face for mask classification
    def _prepare_face(self, face_frame):
        # Resize face to the size model was trained on
        resized_face = cv2.resize(face_frame, (150,150))
        # Normalize pixel values
        normalized_face = resized_face / 255.0
        # Reshape face in a way model expects it
        reshaped_face = np.reshape(normalized_face, (1,150,150,3))
        reshaped_face = np.vstack([reshaped_face])
        
        return reshaped_face

    # Method to prepare face for face detection model
    def _prepare_face_for_face_detection(self, frame):
        # Convert image to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Resize face to the size model was trained on
        face_image = cv2.resize(gray, (64, 64))
        # Reshape and normalize image
        face_image = np.reshape(face_image, (1, 64, 64, 1))
        face_image = face_image / 255.0
        
        return face_image

    # Method to perform face and mask detection in a video feed
    def detect_mask_and_face_in_video_feed(self):
        # Start video capture
        video = cv2.VideoCapture(0)
        
        # Start loop to continuously capture video frames
        while True:
            # Capture video frame
            ret, frame = video.read()
            # Flip frame horizontally
            frame = cv2.flip(frame, 1, 1)
            # Resize frame for face detection
            small_frame = cv2.resize(frame, (frame.shape[1] // self.scale, frame.shape[0] // self.scale))
            # Perform face detection
            faces = self.face_detector.detectMultiScale(small_frame)
            
            # Loop over all detected faces
            for face in faces:
                # Get face coordinates and size
                x, y, w, h = [v * self.scale for v in face]
                # Get face frame
                face_frame = frame[y:y+h, x:x+w]
                # Prepare face for mask classification
                processed_face = self._prepare_face(face_frame)
                
                # Perform mask classification
                pred = self.mask_classifier.predict(processed_face)
                # Get classification result
                pred_result = np.argmax(pred, axis=1)[0]
                
                # Draw rectangle around face based on mask/no mask
                cv2.rectangle(frame, (x,y), (x+w, y+h), self.color_dict[pred_result], 2)
                cv2.rectangle(frame, (x, y-40), (x+w, y), self.color_dict[pred_result], -1)
                cv2.putText(frame, self.label_dict[pred_result], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

                # Prepare face for face detection
                face_frame_face_detection = self._prepare_face_for_face_detection(face_frame)
                # Perform face detection
                face_detection_pred = self.face_classifier.predict(face_frame_face_detection)
                
                # Define text location
                text_location = (frame.shape[1] - 200, 50)  # adjust the values as needed

                # Put text based on face detection result
                if face_detection_pred > 0.5:
                    cv2.putText(frame, 'Face', text_location, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
                else:
                    cv2.putText(frame, 'No Face', text_location, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

            # Show video frame with detections
            cv2.imshow('MASK AND FACE DETECTOR', frame)
            
            # Break the loop on 'ESC' press
            if cv2.waitKey(10) == 27:
                break
                
        # Release video capture object and destroy windows
        video.release()
        cv2.destroyAllWindows()

# Main function to start detection
def main():
    # Create detector object
    detector = MaskAndFaceDetector()
    # Perform detection
    detector.detect_mask_and_face_in_video_feed()

# Call main function
if __name__ == "__main__":
    main()

# Face Detector alone

In [ ]:
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model

# # Load the trained model
# model = load_model('face_detection_model.h5')

# # Function to perform real-time face detection
# def detect_faces(frame):
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     face_image = cv2.resize(gray, (64, 64))
#     face_image = np.reshape(face_image, (1, 64, 64, 1))
#     face_image = face_image / 255.0
#     prediction = model.predict(face_image)
#     if prediction > 0.5:
#         cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 2)
#         cv2.putText(frame, 'Face', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
#     else:
#         cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 2)
#         cv2.putText(frame, 'No Face', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
#     return frame

# # Open the webcam
# video_capture = cv2.VideoCapture(0)

# # Process the video frames in real-time
# while True:
#     # Capture frame-by-frame
#     ret, frame = video_capture.read()
    
#     # Perform face detection
#     output_frame = detect_faces(frame)
    
#     # Display the resulting frame
#     cv2.imshow('Face Detection', output_frame)
    
#     # Exit if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# video_capture.release()
# cv2.destroyAllWindows()
